# Variables

In [2]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

# Setup

In [1]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': '97B6YV2F0AK5KE3C',
  'HostId': 'tFPLEsAXoXk5kDP7fEEQxYHP6yYWMnFHSNKQzWEJd0dmsF/LYwHuO71/rxTSXwi68p5Nf3+WoDo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'tFPLEsAXoXk5kDP7fEEQxYHP6yYWMnFHSNKQzWEJd0dmsF/LYwHuO71/rxTSXwi68p5Nf3+WoDo=',
   'x-amz-request-id': '97B6YV2F0AK5KE3C',
   'date': 'Fri, 14 Mar 2025 22:48:45 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'era5-bra

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-03-14 17:48:52,514][INFO    ][coiled] Fetching latest package priorities...
[2025-03-14 17:48:52,515][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-03-14 17:48:53,453][INFO    ][coiled.package_sync] Scanning 442 conda packages...
[2025-03-14 17:48:53,463][INFO    ][coiled.package_sync] Scanning 257 python packages...
[2025-03-14 17:48:54,547][INFO    ][coiled] Running pip check...
[2025-03-14 17:48:56,229][INFO    ][coiled] Validating environment...
[2025-03-14 17:48:58,261][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-03-14 17:48:58,812][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-03-14 17:48:58,814][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-03-14 17:49:00,610][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-zhsxf.dask.host/3ATxUZ27dRJB0Iu8/status


Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/iostream.py", line 1367, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/ssl.py", line 1319, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLEOFError: [SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 202, in _handle_events
    handler_func(fileobj, events)
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/pyth

In [ ]:
import dask_geopandas as dgpd
import geopandas as gpd
import dask.dataframe as dd

# Define S3 file path
s3_path = "s3://your-bucket-name/combined_cities.parquet"

# S3 path of the zipped Parquet file
s3_zip_path = "s3://your-bucket-name/combined_cities.zip"

urban_extent = f'{GRIDS_PATH}/{city_name}/{city_name}_urban_extent.geoparquet'
extent = dgpd.read_parquet(urban_extent)
parquet_filename_inside_zip = "combined_cities.parquet"  # Name inside the ZIP

# Open the zip file from S3
with fsspec.open(s3_zip_path, mode="rb") as s3_file:
    with zipfile.ZipFile(io.BytesIO(s3_file.read()), "r") as z:
        with z.open(parquet_filename_inside_zip) as parquet_file:
            # Read the Parquet file as a Dask DataFrame
            dask_df = dd.read_parquet(parquet_file, engine="pyarrow")

# Read Parquet file as a Dask DataFrame
dask_df = dd.read_parquet(s3_path, engine="pyarrow", storage_options={"anon": False})

# Convert to Dask GeoDataFrame
dask_gdf = dgpd.from_dask_dataframe(dask_df)


PermissionError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: All access to this object has been disabled

In [7]:
client.close()